# Merge craiglist data with synthetic census data

Steps: 

1. Get craigslist data 
 - Query from remote database in batches based on FIPS
 - Joing to census via FIPS code
2. Get census data 
 - Aggregate to BG
3. Merge together

Can process in batches, one state at a time, or get data by region.

TODO: 
 - The census data for DC are missing. The files are there but they contain no data.
 - If need to look at education or other person data, aggregate person data to household and then block group

In [5]:
import pandas as pd
import psycopg2
import paramiko
import os
import numpy as np
import json

### Craigslist data table columns

pid          | character varying(25)  |
 date         | date                   | 
 region       | character varying(50)  | 
 neighborhood | character varying(200) | 
 rent         | double precision       | 
 bedrooms     | double precision       | 
 sqft         | double precision       | 
 rent_sqft    | double precision       | 
 longitude    | double precision       | 
 latitude     | double precision       | 
 county       | character varying(20)  | 
 fips_block   | character varying(20)  | 
 state        | character varying(20)  | 
 bathrooms

**FIPS code format**

 53-----033---001701--1--015

[state][county][tract][bg][block]

Note: for DC, county='001'

### Synthetic census data - variables

**Household data**: household_id,serialno,persons,cars,income,race_of_head,age_of_head,workers,state,county,tract,block group,children,tenure,recent_mover

**Person data**: person_id,member_id,age,relate,edu,sex,hours,earning,race_id,household_id,student,work_at_home,worker

## Remote connection parameters

Make sure you have two files saved in the same directory, postgres_settings.json and ssh_settings.json. 

In [6]:
DATA_DIR=os.path.join('..','data') 
"""Path to local data directory"""

#read postgres connection parameters
with open('postgres_settings.json') as settings_file:    
    settings = json.load(settings_file)

DBNAME = settings['dbname']
USER = settings['user']
HOST = settings['host']
PASSWORD = settings['password']

conn_str = "dbname = {0} user = {1} host = {2} password = {3}".format(DBNAME, USER, HOST, PASSWORD)

try:
    conn = psycopg2.connect(conn_str)
    cur = conn.cursor()
except:
    print ("Cannot connection. Check settings")


In [7]:
# TODO: add putty connection too. 
#read SSH connection parameters
with open('ssh_settings.json') as settings_file:    
    settings = json.load(settings_file)

HOSTNAME = settings['hostname']
USERNAME = settings['username']
PASSWORD = settings['password']
LOCAL_KEY_DIR = settings['local_key_dir']

CENSUS_DIR = 'synthetic_population'
"""Remote directory with census data"""

RESULTS_DIR = 'craigslist_census'
"""Remote directory for results"""

# estbalish SSH connection
ssh = paramiko.SSHClient() 
ssh.load_host_keys(LOCAL_KEY_DIR)
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(HOSTNAME,username=USERNAME, password=PASSWORD)
sftp = ssh.open_sftp()

### Create FIPS look-up tables

In [8]:
# make dictionary of states and fips codes. 
fips_state = pd.read_csv(os.path.join(DATA_DIR,'state_fips_codes.csv'),dtype=str)
fips2state=dict(zip(fips_state['FIPS'],fips_state['USPS']))
state2fips=dict(zip(fips_state['USPS'],fips_state['FIPS']))

# Make lookup for county to MPO code 
mpo_counties = pd.read_csv(os.path.join(DATA_DIR,'us_2015_mpo_regions_counties_v1.csv'), encoding='latin1', dtype={'MPO_ID':str,'COUNTYFP':str,'STFIPS':str})
mpo_counties['COUNTYFP'] = mpo_counties['COUNTYFP'].str.zfill(2)  
mpo_counties['st_co_fips'] = mpo_counties['STFIPS']+mpo_counties['COUNTYFP']  # we will want to join on 2-char state + 3-char county fips
county2mpo=dict(zip(mpo_counties['st_co_fips'],mpo_counties['MPO_ID']))  # do we want MPO_ID or do we want GEOID? 
mpo_counties.head()

,ID,MPO_ID,MPONAME,STATE,STFIPS,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,...,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PERIMETER,area_sqmi,st_co_fips
0,1826900.0,47198201,Johnson City Metropolitan Transportation Plann...,TN,47,47,163,1639793,47163,Sullivan,...,28700.0,NaN,A,1.070725e+09,4.220920e+07,36.510213,-82.299396,71837.801334,0.756540,47163
1,1541707.0,29201300,Southeast Metropolitan Planning Organization (...,MO,29,17,003,424203,17003,Alexander,...,16020.0,NaN,A,6.099969e+08,4.423719e+07,37.183657,-89.349516,6972.897419,0.591829,29003
2,2101128.0,51197401,Richmond Area MPO,VA,51,51,670,1498428,51670,Hopewell,...,40060.0,NaN,F,2.665508e+07,1.324078e+06,37.291010,-77.298944,6041.591069,0.257294,51670
3,220693.0,15201300,Maui MPO,HI,15,15,009,365283,15009,Maui,...,27980.0,NaN,A,-1.286608e+09,-1.091674e+09,20.855931,-156.601550,299148.486465,728.079214,15009
4,220707.0,15197500,Oahu MPO,HI,15,15,003,365281,15003,Honolulu,...,46520.0,NaN,A,1.555505e+09,-2.400273e+08,21.461365,-158.201974,329335.691836,610.910815,15003


In [9]:
def run_query(q):
    """ Get results given SQL query"""
    cur.execute(q)
    return(cur.fetchall())

def get_craiglist(filters):
    """Get craiglist data from database.
    Args: 
        filters (list): list of strings containing filter criteria. Format as individual SQL WHERE statements. E.g., ["region='sandiego'","rent>100"]
    Returns: 
        DataFrame: listings data. 
    """
    #q="SELECT pid,date,rent,bedrooms,bathrooms,sqft,rent_sqft,fips_block,state,region,longitude,latitude FROM rental_listings WHERE state='{}';".format(state)
    filters_str = ' AND '.join([x for x in filters])
    q="SELECT pid,date,rent,bedrooms,bathrooms,sqft,rent_sqft,fips_block,state,region,longitude,latitude FROM rental_listings WHERE {};".format(filters_str)
    results=run_query(q)
    df=pd.DataFrame(results,columns=['listing_id', 'date','rent','bedrooms','bathrooms','sqft','rent_sqft','fips_block','state','region','lng','lat'] )  # put it all into a dataframe
    
    # split FIPS into different columns - split off the last 3 chars
    df['block']=df.fips_block.str[-4:]
    df['fips12']=df.fips_block.str[:-3]
    return(df)

def read_census_file(fname):
    """Read census csv file via SFTP and return as dataframe."""
    with sftp.open(os.path.join(CENSUS_DIR,fname)) as f:
        df = pd.read_csv(f, delimiter=',',dtype={'age_of_head':float, 'block group':str, 'cars':float, 'children':float, 'county':str,
           'household_id':str, 'income':float, 'persons':float, 'race_of_head':str, 'recent_mover':str,
           'serialno':str, 'state':str, 'tenure':str, 'tract':str, 'workers':float})
    return df

def write_results_file(data,fname):
    """Write merged data to csv file via SFTP."""
    with sftp.open(os.path.join(RESULTS_DIR,fname),'w') as f:
        data.to_csv(f,index=True)
    return

def get_census_by_state(state, table='households'): 
    """Return all census data for state given two-char abbreviation. Can be 'households' or 'persons' data. """ 
    filelist=sftp.listdir(CENSUS_DIR)
    if table=='households':
        files = [f for f in filelist if f[:5]=='hh_{}'.format(state)]
    elif table=='persons':
        files = [f for f in filelist if f[:4]=='p_{}'.format(state)]
    #files = files[:3]  # uncomment this line for testing.
    new_df = pd.DataFrame()
    for f in files:
        df = read_census_file(f)
        new_df = pd.concat([new_df,df])
    return(new_df)

def strip_zeros(s):
    """Remove '.0 from end of string"""
    if s.endswith('.0'):
        return(s[:-2])
    else:
        return(s)

def format_hh_data(df):
    """Fix formatting for hhs census data. Replace '' strings with zero. Format other strings."""

    df['county'] = df['county'].str.zfill(2)  # make county 3-char string.
    
    for col in ['children','workers']:
        df[col] = df[col].replace('','0')

    for col in ['race_of_head','recent_mover','tenure']:
        df[col] = df[col].astype(str)
        df[col] = df[col].map(strip_zeros)  # make sure strings are formatted. 
    return(df)

def aggregate_census(df, groupby_cols=['county','tract','block group'],cols_to_sum=['cars','children','persons','workers'], cols_to_median=['age_of_head','income'],categ_cols=['race_of_head','recent_mover','tenure'],id_col='serialno',table='hhs'):
    """Aggregate census table to block group. Made this for hh data, may need to revised for persons data.
    Args: 
        groupby_cols (list): names of columns to group by (default=['county','tract','block group'])
        cols_to_sum (list): names of columns for which to compute totals. 
        cols_to_median (list): names of columns for which to compute medians
        categ_cols (list): names of categorical columns
        id_col (str): name of column that serves as the id column, to use in counting rows. 
        table (str): 'hhs' (default) or 'per'
    Returns: 
        DataFrame: aggregated data. 
        """
    # For some columns we'll want to find the sum or average/median. These will need only a simple groupby 
    sums = df.groupby(by=groupby_cols).sum()[cols_to_sum]
    sums.columns = [x+'_tot' for x in cols_to_sum]
    
    medians = df.groupby(by=groupby_cols).median()[cols_to_median]
    medians.columns = [x+'_med' for x in cols_to_median]
    
    counts = pd.DataFrame(df.groupby(by=groupby_cols).count()[id_col])
    counts.columns=[table+'_tot']

    # Categorical columns will need pivot tables. 
    categoricals = pd.DataFrame(index=counts.index)
    for col in categ_cols:
        pivoted=df.pivot_table(index = groupby_cols, columns = col, aggfunc='count')[id_col]
        pivoted.columns = [col+'_'+x for x in pivoted.columns]
        pivoted.columns = pivoted.columns.map(strip_zeros)
        # merge back together
        categoricals = pd.merge(categoricals, pivoted, left_index=True, right_index=True)

    # put all back together in one table
    merged = pd.merge(sums, medians, left_index=True, right_index=True)
    merged = pd.merge(merged, counts, left_index=True, right_index=True)
    merged = pd.merge(merged, categoricals, left_index=True, right_index=True)

    # check lengths of dataframes to detect any problems in grouping or merging
    lengths = [len(sums),len(medians),len(counts),len(categoricals),len(merged)]
    if len(set(lengths))>1:
        print('Warning: Aggregated tables have different lengths.',lengths,'for sums, medians, counts, categoricals, and merged.')
    
    return(merged)

def match_mpo(s, mpo_dict=county2mpo):
    """Match a 5-char state-county FIPS code to an MPO code
    Args: 
        s (str): 5-char state-county string
        mpo_dict (dict): county2mpo dictionary
    Returns: 
        str: MPO code
    """
    try: 
        return mpo_dict[s]
    except KeyError: # in this case, the county is not in an MPO
        return ''

In [10]:
def run_all(state, filters=None):
    """Get craigslist data and merge with census data, by state, and save.  with additional filters if needed. 
    Args: 
        state (str): 2-char state abbreviation
        filters (list): additional filters. Do not need to include state in filter list
    """
    
    # load and prepare craiglist data
    
    # If filters are provided, use them to filter data
    if filters:
        filters.append("state='{}'".format(state))
        print(filters)
        df_cl=get_craiglist(filters)
    # If no filters provided, get all data for the specified state. 
    else:
        df_cl=get_craiglist(["state='{}'".format(state)])
    
    df_cl['st_co_fps'] = df_cl.fips_block.map(lambda x: x[:5])
    df_cl['mpo_id'] = df_cl.st_co_fps.map(match_mpo)

    # load and prepare census data for households
    hhs = get_census_by_state(state, table='households')
    hhs = format_hh_data(hhs)
    hhs_bg = aggregate_census(hhs)
    hhs_bg=hhs_bg.reset_index()
    hhs_bg['fips12']=state2fips[state]+hhs_bg['county']+hhs_bg['tract']+hhs_bg['block group'] # create 12-digit FIPS code for merging. 

    # merge with craigslist data. 
    merged = pd.merge(df_cl, hhs_bg, on='fips12',how='left')
    merged = merged.set_index('listing_id')

    #TODO: add persons data here, if needed. 

    # Keep only columns we'll need.
    cols_to_keep=['date','rent','bedrooms','bathrooms','sqft','rent_sqft','fips_block','state','region','mpo_id','lng','lat','cars_tot','children_tot','persons_tot','workers_tot','age_of_head_med','income_med','hhs_tot','race_of_head_1','race_of_head_2','race_of_head_3','race_of_head_4','race_of_head_5','race_of_head_6','race_of_head_7','race_of_head_8','race_of_head_9','recent_mover_0','recent_mover_1','tenure_1','tenure_2']
    
    # This is a bit of a hack in case some columns are missing in some states. 
    for col in cols_to_keep: 
        if col not in merged.columns:
            merged[col] = np.nan

    # save file either locally or remotely. 
    print('Saving data for {s}: {m} rows'.format(s=state,m=len(merged)))
    outfile = 'cl_census_{}.csv'.format(state)
    #merged[cols_to_keep].to_csv(os.path.join(DATA_DIR,outfile), index=True)  # uncomment to save locally

    #write_results_file(merged[cols_to_keep], outfile)  # uncomment to save remotely. 
    return merged[cols_to_keep]

## Get data for a single region

Use this to get data for a single region, for use in our preliminary analysis

In [11]:
df_bayarea = run_all(state='CA',filters=["region = 'sfbay'","rent>0"])   # define whatever filters you want here.
df_bayarea.head()

["region = 'sfbay'", 'rent>0', "state='CA'"]
Saving data for CA: 125539 rows


,date,rent,bedrooms,bathrooms,sqft,rent_sqft,fips_block,state,region,mpo_id,...,race_of_head_4,race_of_head_5,race_of_head_6,race_of_head_7,race_of_head_8,race_of_head_9,recent_mover_0,recent_mover_1,tenure_1,tenure_2
listing_id,,,,,,,,,,,,,,,,,,,,,
5925013098,2016-12-18,2495.0,0.0,NaN,398.0,6.268844,060750162001001,CA,sfbay,06197001,...,1.0,NaN,42.0,2.0,14.0,27.0,276.0,153.0,51.0,378.0
5925007228,2016-12-18,3300.0,2.0,NaN,1357.0,2.431835,060133342005000,CA,sfbay,06197001,...,NaN,NaN,NaN,NaN,NaN,NaN,357.0,72.0,225.0,204.0
5925014745,2016-12-18,1657.0,2.0,NaN,915.0,1.810929,060855091022006,CA,sfbay,06197001,...,NaN,2.0,782.0,NaN,8.0,24.0,707.0,539.0,172.0,1074.0
5921274233,2016-12-18,1700.0,1.0,NaN,500.0,3.400000,060871006002003,CA,sfbay,06197502,...,NaN,NaN,14.0,NaN,NaN,NaN,337.0,68.0,268.0,137.0
5925012514,2016-12-18,2195.0,1.0,NaN,443.0,4.954853,060750125011002,CA,sfbay,06197001,...,NaN,10.0,473.0,8.0,69.0,61.0,1124.0,68.0,2.0,1190.0


In [ ]:
# save locally
outfile = 'sfbay_listings_04092017.csv'
df_bayarea.to_csv(os.path.join(DATA_DIR,outfile), index=True) 

## Process all data by state
Use this to merge all the data by state

In [ ]:
for state in fips_state['USPS']:# uncomment when done with testing. 
    if state != 'DC':   # the DC census data is missing. 
        print('\n Working on',state)
        df_state = run_all(state)

In [ ]:
df_state.head()

In [ ]:
ssh.close()